In [1]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv, set_key

In [2]:
# 当前windows
# 获取当前的 Conda 环境路径
conda_env_path = os.environ.get('CONDA_PREFIX')

# ".env" 文件的绝对路径
dotenv_path = os.path.join(conda_env_path, '.env')

# 加载 ".env" 文件
_ = load_dotenv(dotenv_path, verbose=True)
openai.api_key = os.environ['OPENAI_API_KEY']

# Intro to Prompting module

- Intro to Prompt Engineering: Tips and Tricks: Explore a range of valuable tips and tricks for effective prompting. We will cover techniques such as role prompting, which involves specifying a specific role for the model, such as assistant or copywriter. Additionally, we will delve into few-shot prompting, teaching the model how to respond based on limited examples. Lastly, we will examine the chain of thought approach, which aids in enhancing reasoning capabilities. Throughout the lesson, we will provide numerous examples of successful and ineffective prompts, equipping you with the skills to master the art of crafting optimal prompts for various scenarios.
- Using Prompt Templates: Dynamic prompting provides a flexible approach to improve the model’s context. This lesson offers a comprehensive explanation of few-shot learning, providing compelling examples and allowing the library to select suitable samples based on the input window length of the model.
- Getting the Best of Few Shot Prompts and Example Selectors: Discussing the advantages and disadvantages of few-shot learning, including the enhanced output quality achievable by defining tasks through examples. However, we will also delve into potential drawbacks, such as increased token usage and subpar results when utilizing poorly chosen examples. Furthermore, we will demonstrate how to use example selectors effectively and provide insights into when and why they should be employed.
- Managing Outputs with Output Parsers: Parsing the output is a crucial aspect of interacting with language models. Output parsers offer the flexibility to select from pre-defined types or create a custom data schema, enabling precise control over the output format. Additionally, output fixer classes are crucial in identifying and correcting misformatted responses, ensuring consistent and error-free outputs. These powerful tools are indispensable in a production environment, guaranteeing the reliability and consistency of application outputs.
- Improving Our News Articles Summarizer: This project will leverage the code from the previous module as a foundation and integrate the new concepts introduced in previous lessons. Still, the project's primary objective is to generate summaries of news articles. The process begins by retrieving the content from a given URL. Then, a few-shot prompt template is employed to specify the desired output style. Finally, the output parser transforms the model's string response into a list format, facilitating convenient utilization.
- Creating Knowledge Graphs from Textual Data: Unveiling Hidden Connections: The second project in this module utilizes the text understanding capability of language models to generate knowledge graphs. We can effortlessly extract triple relations from text and format them accordingly using predefined variables within the LangChain library. Furthermore, the option to visualize the knowledge graph enhances comprehension and facilitates easier understanding.

# Intro to Prompt Engineering: Tips and Tricks

## Role Prompting
1. Specify the role in your prompt, e.g., "As a copywriter, create some attention-grabbing taglines for AWS services."
2. 
Use the prompt to generate an output from an LLM
3. 
Analyze the generated response and, if necessary, refine the prompt for better results.

In [3]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
# Initialize LLM
llm = OpenAI(model_name="text-davinci-003", temperature=0)

template = """
As a futuristic robot band conductor, I need you to help me come up with a song title.
What's a cool song title for a song about {theme} in the year {year}?
"""
prompt = PromptTemplate(
    input_variables=["theme", "year"],
    template=template,
)

# Create the LLMChain for the prompt
llm = OpenAI(model_name="text-davinci-003", temperature=0)

# Input data for the prompt
input_data = {"theme": "interstellar travel", "year": "3030"}

# Create LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the LLMChain to get the AI-generated song title
response = chain.run(input_data)

print("Theme: interstellar travel")
print("Year: 3030")
print("AI-generated song title:", response)

Theme: interstellar travel
Year: 3030
AI-generated song title: 
"Journey to the Stars: 3030"


This is a good prompt for several reasons:

- Clear instructions: The prompt is phrased as a clear request for help in generating a song title, and it specifies the context: "As a futuristic robot band conductor." This helps the LLM understand that the desired output should be a song title related to a futuristic scenario.
- Specificity: The prompt asks for a song title that relates to a specific theme and a specific year, "{theme} in the year {year}." This provides enough context for the LLM to generate a relevant and creative output. The prompt can be adapted for different themes and years by using input variables, making it versatile and reusable.
- Open-ended creativity: The prompt allows for open-ended creativity, as it doesn't limit the LLM to a particular format or style for the song title. The LLM can generate a diverse range of song titles based on the given theme and year.
- Focus on the task: The prompt is focused solely on generating a song title, making it easier for the LLM to provide a suitable output without getting sidetracked by unrelated topics.

## Few Shot Prompting

In [4]:
from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model_name="text-davinci-003", temperature=0)

examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}\n
"""
example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Here are some examples of colors and the emotions associated with them:\n\n",
    suffix="\n\nNow, given a new color, identify the emotion associated with it:\n\nColor: {input}\nEmotion:",
    input_variables=["input"],
    example_separator="\n",
)

formatted_prompt = few_shot_prompt.format(input="purple")

# Create the LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=PromptTemplate(template=formatted_prompt, input_variables=[]))

# Run the LLMChain to get the AI-generated emotion associated with the input color
response = chain.run({})

print("Color: purple")
print("Emotion:", response)

Color: purple
Emotion:  creativity


### Bad Prompt Practices

In [ ]:
from langchain import PromptTemplate

template = "Tell me something about {topic}."
prompt = PromptTemplate(
    input_variables=["topic"],
    template=template,
)
prompt.format(topic="dogs")

## Chain Prompting

To use chain prompting with LangChain, you could:

- Extract relevant information from the generated response.
- Use the extracted information to create a new prompt that builds upon the previous response.
- Repeat steps as needed until the desired output is achieved.

In [5]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model_name="text-davinci-003", temperature=0)

# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Provide a brief description of {scientist}'s theory of general relativity.
Answer: """
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Extract the scientist's name from the response
scientist = response_question.strip()

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)

Scientist: Albert Einstein
Fact: 
Albert Einstein's theory of general relativity is a theory of gravitation that states that the gravitational force between two objects is a result of the curvature of spacetime caused by the presence of mass and energy. It explains the phenomenon of gravity as a result of the warping of space and time by matter and energy.


### Bad Prompt Example:

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model_name="text-davinci-003", temperature=0)

# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Tell me something interesting about {scientist}.
Answer: """
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Extract the scientist's name from the response
scientist = response_question.strip()

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)

### An example of the unclear prompt:

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model_name="text-davinci-003", temperature=0)

# Prompt 1
template_question = """What are some musical genres?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Tell me something about {genre1}, {genre2}, and {genre3} without giving any specific details.
Answer: """
prompt_fact = PromptTemplate(input_variables=["genre1", "genre2", "genre3"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Assign three hardcoded genres
genre1, genre2, genre3 = "jazz", "pop", "rock"

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"genre1": genre1, "genre2": genre2, "genre3": genre3}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Genres:", genre1, genre2, genre3)
print("Fact:", response_fact)

## Chain of Thought Prompting

Chain of Thought Prompting (CoT) is a technique developed to encourage large language models to explain their reasoning process, leading to more accurate results. By providing few-shot exemplars demonstrating the reasoning process, the LLM is guided to explain its reasoning when answering the prompt. This approach has been found effective in improving results on tasks like arithmetic, common sense, and symbolic reasoning.

### Tips for Effective Prompt Engineering
- Be specific with your prompt: Provide enough context and detail to guide the LLM toward the desired output.
- Force conciseness when needed.
- Encourage the model to explain its reasoning: This can lead to more accurate results, especially for complex tasks.

In [6]:
from langchain import FewShotPromptTemplate, PromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model_name="text-davinci-003", temperature=0)

examples = [
    {
        "query": "What's the secret to happiness?",
        "answer": "Finding balance in life and learning to enjoy the small moments."
    }, {
        "query": "How can I become more productive?",
        "answer": "Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
life coach. The assistant provides insightful and practical advice to the users' questions. Here are some
examples: 
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# Create the LLMChain for the few-shot prompt template
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

# Define the user query
user_query = "What are some tips for improving communication skills?"

# Run the LLMChain for the user query
response = chain.run({"query": user_query})

print("User Query:", user_query)
print("AI Response:", response)

User Query: What are some tips for improving communication skills?
AI Response:  Practice active listening, be mindful of your body language, and be open to constructive feedback.


This prompt:

- Provides a clear context in the prefix: The prompt states that the AI is a life coach providing insightful and practical advice. This context helps guide the AI's responses and ensures they align with the intended purpose.
- Uses examples that demonstrate the AI's role and the type of responses it generates: By providing relevant examples, the AI can better understand the style and tone of the responses it should produce. These examples serve as a reference for the AI to generate similar responses that are consistent with the given context.
- Separates examples and the actual query: This allows the AI to understand the format it should follow, ensuring a clear distinction between example conversations and the user's input. This separation helps the AI to focus on the current query and respond accordingly.
- Includes a clear suffix that indicates where the user's input goes and where the AI should provide its response: The suffix acts as a cue for the AI, showing where the user's query ends and the AI's response should begin. This structure helps maintain a clear and consistent format for the generated responses.

# Using Prompt Templates

# Getting the Best of Few Shot Prompts and Example Selectors

# Managing Outputs with Output Parsers

# Improving Our News Articles Summarizer

# Creating Knowledge Graphs from Textual Data: Unveiling Hidden Connections